In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
def esa_score(phi, alpha):
    #dot product, weighted sum to calc esa
    return np.dot(alpha, phi)

#
def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

In [4]:
df6 = pd.read_csv('credit_risk_dataset - ethics.csv')

# Initialize LabelEncoder
label_encoder = LabelEncoder() 

# Apply label encoding
df6['person_home_ownership'] = label_encoder.fit_transform(df6['person_home_ownership'])
df6['loan_intent'] = label_encoder.fit_transform(df6['loan_intent'])
df6['loan_grade'] = label_encoder.fit_transform(df6['loan_grade'])
df6['cb_person_default_on_file'] = label_encoder.fit_transform(df6['cb_person_default_on_file'])

y = df6['loan_status']
X = df6.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])
esa_features = df6[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df6['CST'].values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

dnn_baseline_preds = np.zeros(len(X))
dnn_override_preds = np.zeros(len(X))
dnn_penalized_preds = np.zeros(len(X))

def build_dnn_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [5]:
b_time = 0
o_time = 0
p_time = 0

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    b_time_s = time.perf_counter()
    # -------- Baseline --------
    model_baseline = build_dnn_model(X_train.shape[1])
    model_baseline.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0,
                           validation_data=(X_val, y_val),
                           callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred_baseline = (model_baseline.predict(X_val).flatten() >= 0.5).astype(int)
    dnn_baseline_preds[val_idx] = y_val_pred_baseline
    print(f"Fold {fold+1} - DNN Baseline:")
    evaluate_classification(y_val, y_val_pred_baseline)
    b_time_e = time.perf_counter()
    b_time = b_time + (b_time_e - b_time_s)

    o_time_s = time.perf_counter()
    # -------- Override --------
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_preds = (esa_vals >= tau_val).astype(int)
    dnn_override_preds[val_idx] = moral_preds
    print(f"Fold {fold+1} - DNN ESA Override:")
    evaluate_classification(y_val, moral_preds)
    o_time_e = time.perf_counter()
    o_time = o_time + (o_time_e - o_time_s)

    p_time_s = time.perf_counter()
    # -------- Penalized --------
    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]
    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    sample_weights = np.clip(1 + 5 * moral_penalty, 1, 10)

    model_penalized = build_dnn_model(X_train.shape[1])
    model_penalized.fit(X_train, y_train, sample_weight=sample_weights, epochs=30, batch_size=32, verbose=0,
                        validation_data=(X_val, y_val),
                        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred_penalized = (model_penalized.predict(X_val).flatten() >= 0.5).astype(int)
    dnn_penalized_preds[val_idx] = y_val_pred_penalized
    print(f"Fold {fold+1} - DNN ESA Penalized:")
    evaluate_classification(y_val, y_val_pred_penalized)
    p_time_e = time.perf_counter()
    p_time = p_time + (p_time_e - p_time_s)

print("\n--- Final Evaluation (DNN Baseline) ---")
#evaluate_classification(y, dnn_baseline_preds)
metrics = evaluate_classification(y, dnn_baseline_preds)
print(metrics)

print("\n--- Final Evaluation (DNN ESA Override) ---")
#evaluate_classification(y, dnn_override_preds)
metrics = evaluate_classification(y, dnn_override_preds)
print(metrics)

print("\n--- Final Evaluation (DNN ESA Penalized) ---")
#evaluate_classification(y, dnn_penalized_preds)
metrics = evaluate_classification(y, dnn_penalized_preds)
print(metrics)

print("time baseline:", b_time)
print("time override:", o_time)
print("time penalized:", p_time)

204/204 [==============================] - 0s 874us/step
Fold 1 - DNN Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      5095
           1       0.92      0.66      0.76      1422

    accuracy                           0.91      6517
   macro avg       0.91      0.82      0.86      6517
weighted avg       0.91      0.91      0.91      6517


Confusion Matrix:
[[5010   85]
 [ 489  933]]
Fold 1 - DNN ESA Override:

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      5095
           1       0.31      0.33      0.32      1422

    accuracy                           0.69      6517
   macro avg       0.56      0.56      0.56      6517
weighted avg       0.70      0.69      0.70      6517


Confusion Matrix:
[[4049 1046]
 [ 957  465]]
204/204 [==============================] - 0s 1ms/step
Fold 1 - DNN ESA Penalized:

Classification 

# CVAR

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import time

In [7]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [8]:
def evaluate_classification(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_pred)
    }

In [9]:
class CVaRLoss(Loss):
    def __init__(self, alpha, phi_tensor, tau_tensor, lam=0.8):
        #initialize main variables
        super().__init__()
        self.alpha = K.constant(alpha, dtype='float32')
        self.phi_tensor = K.constant(phi_tensor, dtype='float32')
        self.tau_tensor = K.constant(tau_tensor, dtype='float32')
        self.lam = lam

    def call(self, y_true, y_pred):
        #implement EMS
        y_true = K.cast(y_true, dtype='float32')
        #regular loss
        bce = K.binary_crossentropy(y_true, y_pred)
        #esa calc
        esa = K.sum(self.phi_tensor * self.alpha, axis=1)
        # Penalize low ESA values below CST threshold (penalty = 0 if esa>=cst, positive otherwise)
        tail_penalty = K.relu(self.tau_tensor - esa)
        return bce + self.lam * tail_penalty

In [10]:
#baseline dnn
def build_dnn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

In [11]:
df7 = pd.read_csv('credit_risk_dataset - ethics.csv')

# Initialize LabelEncoder
label_encoder = LabelEncoder() 

# Apply label encoding
df7['person_home_ownership'] = label_encoder.fit_transform(df7['person_home_ownership'])
df7['loan_intent'] = label_encoder.fit_transform(df7['loan_intent'])
df7['loan_grade'] = label_encoder.fit_transform(df7['loan_grade'])
df7['cb_person_default_on_file'] = label_encoder.fit_transform(df7['cb_person_default_on_file'])

y = df7['loan_status']
X = df7.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])
esa_features = df7[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
phi = df7[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].astype('float32').values

tau = df7['CST'].values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1], dtype='float32')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [12]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
dnn_cvar_preds = np.zeros(len(X))

In [14]:
#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    phi_train = phi[train_idx]
    tau_train = tau[train_idx]

    model = build_dnn(X_train.shape[1])
    model.compile(optimizer=Adam(0.001), loss=CVaRLoss(alpha, phi_train, tau_train, lam=1.0))
    model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred = (model.predict(X_val).flatten() >= 1).astype(int)
    dnn_cvar_preds[val_idx] = y_val_pred
    print(f"Fold {fold+1} - DNN CVaR Loss:")
    print(evaluate_classification(y_val, y_val_pred))
    
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y_val, y_val_pred)
print(metrics)

204/204 [==============================] - 0s 2ms/step
Fold 1 - DNN CVaR Loss:
{'Accuracy': 0.7948442534908701, 'Precision': 1.0, 'Recall': 0.05977496483825598, 'F1': 0.11280690112806901, 'ROC_AUC': 0.529887482419128}
204/204 [==============================] - 0s 1ms/step
Fold 2 - DNN CVaR Loss:
{'Accuracy': 0.7920503376304481, 'Precision': 1.0, 'Recall': 0.047116736990154715, 'F1': 0.08999328408327738, 'ROC_AUC': 0.5235583684950773}


204/204 [==============================] - 0s 1ms/step
Fold 3 - DNN CVaR Loss:
{'Accuracy': 0.7879066912216084, 'Precision': 1.0, 'Recall': 0.02812939521800281, 'F1': 0.05471956224350205, 'ROC_AUC': 0.5140646976090014}
204/204 [==============================] - 0s 1ms/step
Fold 4 - DNN CVaR Loss:
{'Accuracy': 0.8040208717004297, 'Precision': 1.0, 'Recall': 0.10133708655876143, 'F1': 0.18402555910543128, 'ROC_AUC': 0.5506685432793808}


204/204 [==============================] - 0s 1ms/step
Fold 5 - DNN CVaR Loss:
{'Accuracy': 0.80463474524248, 'Precision': 1.0, 'Recall': 0.10415200562983815, 'F1': 0.18865519439133205, 'ROC_AUC': 0.5520760028149191}
Time taken: 650.198107 seconds
{'Accuracy': 0.80463474524248, 'Precision': 1.0, 'Recall': 0.10415200562983815, 'F1': 0.18865519439133205, 'ROC_AUC': 0.5520760028149191}


# Adversarial Formulation

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import time

In [29]:
def esa_score(phi, alpha):
    return np.dot(phi, alpha)

def evaluate_classification(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_pred)
    }

In [24]:
def adversary_loss(esa_true, esa_pred):
    return K.mean(K.square(esa_true - esa_pred))

In [25]:
def build_esa_adversary(input_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(32, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(16, activation='relu')(x)
    esa_output = Dense(1, activation='linear', name='esa_output')(x)
    return Model(inputs=inputs, outputs=esa_output)

In [26]:
def build_adversarial_model(input_dim):
    # Main classifier
    input_layer = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(input_layer)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_class = Dense(1, activation='sigmoid', name='class_output')(x)

    # ESA adversary
    x_esa = Dense(16, activation='relu')(x)
    output_esa = Dense(1, activation='linear', name='esa_output')(x_esa)

    model = Model(inputs=input_layer, outputs=[output_class, output_esa])
    return model

In [30]:
df = pd.read_csv('credit_risk_dataset - ethics.csv')

# Initialize LabelEncoder
label_encoder = LabelEncoder() 

# Apply label encoding
df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])

y = df['loan_status']
X = df.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])

phi = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].astype('float32').values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1], dtype='float32')
ESA_target = esa_score(phi, alpha).reshape(-1, 1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [31]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
dnn_adv_preds = np.zeros(len(X))


In [32]:
#start timer
start_time = time.perf_counter()

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    ESA_train, ESA_val = ESA_target[train_idx], ESA_target[val_idx]

    model = build_adversarial_model(X_train.shape[1])
    model.compile(optimizer=Adam(0.001),
                  loss={'class_output': 'binary_crossentropy', 'esa_output': 'mse'},
                  loss_weights={'class_output': 1.0, 'esa_output': 0.5})

    model.fit(X_train, {'class_output': y_train, 'esa_output': ESA_train},
              epochs=30, batch_size=32, verbose=0,
              validation_split=0.1,
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

    y_val_pred = (model.predict(X_val)[0].flatten() >= 0.5).astype(int)
    dnn_adv_preds[val_idx] = y_val_pred
    print(f"Fold {fold+1} - ESA Adversarial:")
    print(evaluate_classification(y_val, y_val_pred))
    
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y_val, y_val_pred)
print(metrics)

204/204 [==============================] - 0s 786us/step
Fold 1 - ESA Adversarial:
{'Accuracy': 0.9053245358293693, 'Precision': 0.9197080291970803, 'Recall': 0.620253164556962, 'F1': 0.7408651826963462, 'ROC_AUC': 0.8025701544080198}
204/204 [==============================] - 0s 850us/step
Fold 2 - ESA Adversarial:
{'Accuracy': 0.9165131982811541, 'Precision': 0.9507186858316222, 'Recall': 0.6511954992967651, 'F1': 0.7729549248747913, 'ROC_AUC': 0.8208863244422577}
204/204 [==============================] - 0s 896us/step
Fold 3 - ESA Adversarial:
{'Accuracy': 0.907305095150399, 'Precision': 0.9225206611570248, 'Recall': 0.6279887482419128, 'F1': 0.7472803347280335, 'ROC_AUC': 0.8066327722363863}
204/204 [==============================] - 0s 1ms/step
Fold 4 - ESA Adversarial:
{'Accuracy': 0.9102209944751382, 'Precision': 0.9106090373280943, 'Recall': 0.6523574947220268, 'F1': 0.7601476014760147, 'ROC_AUC': 0.8172484235141046}
204/204 [==============================] - 0s 1ms/step
Fold 